# Week05 Online Class
(21.10.05 12:00 Gitter)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#filter" data-toc-modified-id="filter-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>filter</a></span></li><li><span><a href="#불용어" data-toc-modified-id="불용어-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>불용어</a></span></li><li><span><a href="#파이프라인" data-toc-modified-id="파이프라인-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>파이프라인</a></span></li><li><span><a href="#파일에-쓰기" data-toc-modified-id="파일에-쓰기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>파일에 쓰기</a></span></li><li><span><a href="#groupBy" data-toc-modified-id="groupBy-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>groupBy</a></span></li><li><span><a href="#Paired-RDD" data-toc-modified-id="Paired-RDD-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Paired RDD</a></span><ul class="toc-item"><li><span><a href="#groupByKey" data-toc-modified-id="groupByKey-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>groupByKey</a></span></li></ul></li><li><span><a href="#그래프" data-toc-modified-id="그래프-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>그래프</a></span></li><li><span><a href="#DF" data-toc-modified-id="DF-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>DF</a></span><ul class="toc-item"><li><span><a href="#타입-가져오기" data-toc-modified-id="타입-가져오기-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>타입 가져오기</a></span></li><li><span><a href="#RRD에서-DF-만들기" data-toc-modified-id="RRD에서-DF-만들기-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>RRD에서 DF 만들기</a></span></li><li><span><a href="#스키마-정의" data-toc-modified-id="스키마-정의-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>스키마 정의</a></span></li><li><span><a href="#csv에서-읽기" data-toc-modified-id="csv에서-읽기-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>csv에서 읽기</a></span></li><li><span><a href="#csv함수로-tsv-읽기" data-toc-modified-id="csv함수로-tsv-읽기-8.5"><span class="toc-item-num">8.5&nbsp;&nbsp;</span>csv함수로 tsv 읽기</a></span></li><li><span><a href="#json-읽기" data-toc-modified-id="json-읽기-8.6"><span class="toc-item-num">8.6&nbsp;&nbsp;</span>json 읽기</a></span></li></ul></li></ul></div>

## filter
`myRdd2.filter(lambda line: u"스파크" in line)`
-> `스파크가 있는 line의 RDD`
- `u스파크`의 `u` = 유니코드

## 불용어
자연어 처리할 때 의미가 없어 제거하는 단어
- 불용어 정의

`stopwords = ['is','am','are','the','for','a', 'an', 'at']`
- 불용어를 filter로 가려내기
```
myRdd_stop = myRdd2.flatMap(lambda x:x.split())\
                    .filter(lambda x: x not in stopwords)
```

## 파이프라인
연속적으로 명령어를 연결하여 실행하는 것
`upper2list=wordsRdd.map(lambda x: [i.upper() for i in x]).collect()` < X
```
wordsLength = wordsRdd\
    .map(len)\
    .collect()
```
< O

## 파일에 쓰기
`spark.sparkContext.parallelize(upper2list).saveAsTextFile(os.path.join("data", "ds_spark_wiki_out2"))`

오류가 있으면 `winutils.exe` 설치

## groupBy
- `groupByKey()`와 비교하여 상대적으로 빠르지 않다
- `groupBy` : unpaired
- `groupByKey` : paired
```
_testList=[("Seoul",1),("Seoul",1),("Seoul",1),("Busan",1),("Busan",1),
           ("Seoul",1),("Busan",1),
           ("Seoul",1),("Seoul",1),("Busan",1),("Busan",1)]
```

-> paired (데이터는 리스트가 밖에 있고, 안에는 튜플로 구성됨 : 튜플의 리스트(2차원))

RDD를 구성하려면 : 
`_testRdd=spark.sparkContext.parallelize(_testList)`

- groupby는 인자가 있다. groupbykey는 인자가 없다

`_testRdd.groupBy(lambda x:x[0]).collect()`

-> Resultlterable(그룹만 만듬)

- 결과 출력

`_testRdd.groupBy(lambda x:x[0]).mapValues(lambda x: list(x)).collect()` mapValues()사용

or

`_testRdd.groupBy(lambda x:x[0]).mapValues(list).collect()` mapVaules() 안에는 함수가 들어감

두번째 list는 함수다

## Paired RDD

```
_testList=[("key1",1),("key1",1),("key1",1),("key2",1),("key2",1),
           ("key1",1),("key2",1),
           ("key1",1),("key1",1),("key2",1),("key2",1)]
```

byKey 함수는 Pair RDD로 구성을 해 놓아야 한다.

- RDD 생성

`_testRdd=spark.sparkContext.parallelize(_testList, 2)` 

2 : 파티션 수 (데이터가 적으면 별의미 없음)

### groupByKey
`_testRdd.groupByKey().collect()`

-> ResultIterable

결과를 보려면`mapValues()`사용

`_testRdd.groupByKey().mapValues(list).collect()`

groupby하고 출력하려면 `mapValues()`가 필요

```
# 단어세기
wc=myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .groupByKey()\
    .mapValues(sum)
```

reduceByKey는 `mapValues()`가 필요하지 않다

```
# 단어세기
myRdd2\
    .flatMap(lambda x:x.split())\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)
```

`_testRdd.reduceByKey(lambda x,y:x+y).collect()`

## 그래프
```
import matplotlib.pyplot as plt
plt.rc('font', family='NanumMyeongjo')       # 폰트 깨짐
plt.barh(range(len(v)), v, color = 'black')  # x는 v (밸류)로 설정
plt.yticks(range(len(v)), k)                 # y는 k (키)로 설정
plt.show()
```

## DF
- df : 구조적 데이터 (ex 엑셀)
- RDD : 비구조적 데이터

스키마 정해서 만들기
### 타입 가져오기
`from pyspark.sql.types import *`
### RRD에서 DF 만들기
`myRdd=spark.sparkContext.parallelize([(1, 'kim', 50.0), (2, 'lee', 60.0), (3, 'park', 70.0)])`
### 스키마 정의
```
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("height", DoubleType(), True)
])
```

`_myDf = spark.createDataFrame(myRdd, schema)`
### csv에서 읽기
```
df = spark\
        .read\
        .format('com.databricks.spark.csv')\
        .options(header='true', inferschema='true', delimiter=',')\
        .load(os.path.join('data','ds_spark.csv'))
```
### csv함수로 tsv 읽기
```
tDf = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\t')\
    .csv(os.path.join('data', 'ds_spark_heightweight.txt'))
tDf.show()
```
### json 읽기
```
jfile= os.path.join('src','ds_twitter_seoul_3.json')
tweetDf= spark.read.json(jfile)
```